In [0]:
# provider-notebooks/Module 3 - Delta Sharing Architectural Patterns and Best Practices/Includes/Demo-Setup-3.2.ipynb

In [0]:
%run ./_common

In [0]:
# Set variables in python and sql
config_values = [
    ("catalog", "global_retail"),
    ("schema", "sales"),
    ("base_table_for_dynamic_view", "customer_orders"),
    ("dynamic_view", "regional_orders_view"),
    ("dynamic_view_share", "regional_orders_dynamic"),
    ("base_table_for_partition_filtering", "orders_by_region"),
    ("us_partition_filtered_share", "us_orders_share"),
    ("eu_partition_filtered_share", "eu_orders_share"),
    ("us_recipient", "us_partner"),
    ("eu_recipient", "eu_partner"),
    ("apac_recipient", "apac_partner"),
]
DA = create_da_objects(config_values)

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS IDENTIFIER(DA.catalog)
COMMENT 'Catalog for fine-grained access control demo';
USE CATALOG IDENTIFIER(DA.catalog);

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS IDENTIFIER(DA.schema)
COMMENT 'Schema for fine-grained access control demo';
USE SCHEMA IDENTIFIER(DA.schema);

In [0]:
# Cleanup existing shares and recipients before setup
cleanup_commands = [
    "DROP RECIPIENT IF EXISTS us_partner",
    "DROP RECIPIENT IF EXISTS eu_partner",
    "DROP RECIPIENT IF EXISTS apac_partner",
    "DROP SHARE IF EXISTS regional_orders_dynamic",
    "DROP SHARE IF EXISTS us_orders_share", 
    "DROP SHARE IF EXISTS eu_orders_share", 
]

for cmd in cleanup_commands:
    try:
        spark.sql(cmd)
    except Exception as e:
        print(f"Note: {cmd} - {str(e)}")

### Create and Populate Table for Dynamic View Demo
Create and populate the table for the Dynamic View demo

In [0]:
%sql
-- Create source table with customer orders
DROP TABLE IF EXISTS global_retail.sales.customer_orders;

CREATE OR REPLACE TABLE global_retail.sales.customer_orders (
  order_id STRING,
  customer_name STRING,
  email STRING,
  region STRING,
  product STRING,
  amount DECIMAL(10,2),
  order_date DATE
)
USING DELTA
COMMENT 'Customer orders across all regions';

In [0]:
%sql
-- Insert sample data
INSERT INTO global_retail.sales.customer_orders VALUES
  -- North America
  ('ORD001', 'John Smith', 'john@example.com', 'United States', 'Laptop', 1200.00, '2025-01-15'),
  ('ORD002', 'Mary Johnson', 'mary@example.com', 'United States', 'Monitor', 350.00, '2025-01-16'),
  
  -- Europe  
  ('ORD003', 'Hans Mueller', 'hans@example.com', 'Europe', 'Keyboard', 89.99, '2025-01-15'),
  ('ORD004', 'Sophie Martin', 'sophie@example.com', 'Europe', 'Mouse', 45.00, '2025-01-17'),
  
  -- Asia Pacific
  ('ORD005', 'Li Wei', 'li@example.com', 'Asia Pacific', 'Tablet', 599.00, '2025-01-16'),
  ('ORD006', 'Yuki Tanaka', 'yuki@example.com', 'Asia Pacific', 'Headphones', 199.00, '2025-01-18');


### Create and Populate Table for Partition Filtering Demo
Create and populate the table for the Partition Filtering demo

In [0]:
%sql
-- Create partitioned table
DROP TABLE IF EXISTS global_retail.sales.orders_by_region;

CREATE OR REPLACE TABLE global_retail.sales.orders_by_region (
  order_id STRING,
  customer_name STRING,
  product STRING,
  amount DECIMAL(10,2),
  order_date DATE,
  region STRING
)
USING DELTA
PARTITIONED BY (region)
COMMENT 'Orders partitioned by region for efficient filtering';

In [0]:
%sql
-- Insert data (automatically partitioned by region)
INSERT INTO global_retail.sales.orders_by_region VALUES
  ('ORD001', 'John Smith', 'Laptop', 1200.00, '2025-01-15', 'United States'),
  ('ORD002', 'Mary Johnson', 'Monitor', 350.00, '2025-01-16', 'United States'),
  ('ORD003', 'Hans Mueller', 'Keyboard', 89.99, '2025-01-15', 'Europe'),
  ('ORD004', 'Sophie Martin', 'Mouse', 45.00, '2025-01-17', 'Europe'),
  ('ORD005', 'Li Wei', 'Tablet', 599.00, '2025-01-16', 'Asia Pacific'),
  ('ORD006', 'Yuki Tanaka', 'Headphones', 199.00, '2025-01-18', 'Asia Pacific');

In [0]:
display_config_values(config_values)